# Feature Engineering & Data Cleaning

In [7]:
# Imports

## Data Manipulation
import numpy as np
import pandas as pd

# Statistics
from sklearn.preprocessing import MinMaxScaler

# Ignore Warning
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [8]:
# Package versions
%reload_ext watermark
%watermark -a "Cézar Mendes" --iversions

Author: Cézar Mendes

numpy : 1.24.3
sys   : 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
pandas: 1.5.3



## Loading Data

In [9]:
df = pd.read_csv('../data/Prep_BankCustomerChurn.csv')
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


# 

# Data Cleaning

In [10]:
df = df.drop(columns = 'customer_id')
df.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


# Outliers

In [18]:
num_list = df.select_dtypes(['float64', 'int64']).columns.tolist()
num_list

['credit_score',
 'gender',
 'age',
 'tenure',
 'balance',
 'products_number',
 'credit_card',
 'active_member',
 'estimated_salary',
 'churn']

In [19]:
# Describe
def count_outliers(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   out = ((df < (q1 - 1.5 * IQR)) | (df > (q3 + 1.5 * IQR))).sum()
   return out

# Skew
def calc_skew(df):
    return [df[col].skew() for col in df]


# Summary Table
def getOverview(df):
    min = df.min()
    Q1 = df.quantile(0.25)
    mean = np.mean(df)
    median = df.quantile(0.5)
    Q3 = df.quantile(0.75)
    max = df.max()
    IQR = Q3 - Q1
    skew = calc_skew(df)
    outliers = count_outliers(df)
    cut_off = IQR * 1.5
    lower, upper = Q1 - cut_off, Q3 + cut_off
        
    

    new_columns = ['Columns name', 'Min', 'Q1','Mean', 'Median', 'Q3', 'Max', 'IQR', 'Lower fence', 'Upper fence', 'Skew', 'Num_Outliers']
        
    data = zip([column for column in df], min, Q1, mean,  median, Q3, max, IQR, lower, upper, skew, outliers)

    new_df = pd.DataFrame(data = data, columns = new_columns)
        
    new_df.set_index('Columns name', inplace = True)
    \
    return new_df.sort_values('Num_Outliers', ascending = False).transpose()

getOverview(df[num_list]).round(2)

Columns name,churn,age,products_number,credit_score,gender,tenure,balance,credit_card,active_member,estimated_salary
Min,0.00,18.00,1.00,350.00,0.00,0.00,0.00,0.00,0.00,11.58
Q1,0.00,32.00,1.00,584.00,0.00,3.00,0.00,0.00,0.00,51002.11
Mean,0.20,38.92,1.53,650.53,0.55,5.01,76485.89,0.71,0.52,100090.24
Median,0.00,37.00,1.00,652.00,1.00,5.00,97198.54,1.00,1.00,100193.92
Q3,0.00,44.00,2.00,718.00,1.00,7.00,127644.24,1.00,1.00,149388.25
Max,1.00,92.00,4.00,850.00,1.00,10.00,250898.09,1.00,1.00,199992.48
IQR,0.00,12.00,1.00,134.00,1.00,4.00,127644.24,1.00,1.00,98386.14
Lower fence,0.00,14.00,-0.50,383.00,-1.50,-3.00,-191466.36,-1.50,-1.50,-96577.10
Upper fence,0.00,62.00,3.50,919.00,2.50,13.00,319110.60,2.50,2.50,296967.45
Skew,1.47,1.01,0.75,-0.07,-0.18,0.01,-0.14,-0.90,-0.06,0.00


In [21]:
# Remove
def remove_outliers(df, columns):
    
    for col in columns:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off
        
        # Identify outlier rows and drop them from the original DataFrame
        df.drop(df[(df[col] > upper) | (df[col] < lower)].index, inplace=True)

remove_outliers(df, df[['age', 'credit_score', 'products_number']])


In [22]:
getOverview(df[['age', 'credit_score', 'products_number']])

Columns name,age,credit_score,products_number
Min,18.000000,386.000000,1.000000
Q1,31.000000,584.000000,1.000000
Mean,37.332199,650.758826,1.516908
Median,37.000000,652.000000,1.000000
Q3,42.000000,717.000000,2.000000
Max,59.000000,850.000000,3.000000
IQR,11.000000,133.000000,1.000000
Lower fence,14.500000,384.500000,-0.500000
Upper fence,58.500000,916.500000,3.500000
Skew,0.361566,-0.042774,0.404561


In [23]:
# Replace Outliers
def replace_outliers_with_fences(df, columns):
    
    for col in columns:
        Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        IQR = Q3 - Q1
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off

        # Replace outlier values with the respective fences
        df[col] = np.where(df[col] > upper, upper, df[col])
        df[col] = np.where(df[col] < lower, lower, df[col])


replace_outliers_with_fences(df, df[['age']])


In [24]:
getOverview(df[['age', 'credit_score', 'products_number']])

Columns name,age,credit_score,products_number
Min,18.000000,386.000000,1.000000
Q1,31.000000,584.000000,1.000000
Mean,37.328903,650.758826,1.516908
Median,37.000000,652.000000,1.000000
Q3,42.000000,717.000000,2.000000
Max,58.500000,850.000000,3.000000
IQR,11.000000,133.000000,1.000000
Lower fence,14.500000,384.500000,-0.500000
Upper fence,58.500000,916.500000,3.500000
Skew,0.355922,-0.042774,0.404561


# Enconding

### One-Hot Encoding

In [25]:
df = pd.concat([df.drop('country', axis = 1),pd.get_dummies(df.country).add_prefix('country_')], axis = 1)
df = pd.concat([df.drop('tenure', axis = 1),pd.get_dummies(df.tenure).add_prefix('tenure_')], axis = 1)
df = pd.concat([df.drop('products_number', axis = 1),pd.get_dummies(df.products_number).add_prefix('products_number_')], axis = 1)

In [26]:
df.head()

,credit_score,gender,age,balance,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,...,tenure_4,tenure_5,tenure_6,tenure_7,tenure_8,tenure_9,tenure_10,products_number_1,products_number_2,products_number_3
0,619,0,42.0,0.00,1,1,101348.88,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,608,0,41.0,83807.86,0,1,112542.58,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,502,0,42.0,159660.80,1,0,113931.57,1,1,0,...,0,0,0,0,1,0,0,0,0,1
3,699,0,39.0,0.00,0,0,93826.63,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,850,0,43.0,125510.82,1,1,79084.10,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Scaling

## MinMaxScaler

In [27]:
columns_to_scale = ["credit_score", "age",
                    "balance", "estimated_salary"]

scaler = MinMaxScaler(clip=True)
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

df.head()

,credit_score,gender,age,balance,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,...,tenure_4,tenure_5,tenure_6,tenure_7,tenure_8,tenure_9,tenure_10,products_number_1,products_number_2,products_number_3
0,0.502155,0,0.592593,0.000000,1,1,0.506735,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0.478448,0,0.567901,0.334031,0,1,0.562709,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.250000,0,0.592593,0.636357,1,0,0.569654,1,1,0,...,0,0,0,0,1,0,0,0,0,1
3,0.674569,0,0.518519,0.000000,0,0,0.469120,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1.000000,0,0.617284,0.500246,1,1,0.395400,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Saving Object

In [28]:
df.to_csv("../data/bankcustomer_clean.csv", index=False)